In [1]:
#### IMPORTS ####

# Visualisation et manipulation des données
from collections import Counter
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

# Fichiers
import json
import csv
import os
import pickle
import zipfile

# Apprentissage
from sklearn.decomposition import NMF
from sklearn.manifold import TSNE as tsne
from sklearn.neighbors import NearestNeighbors

# Interface
from ipywidgets import Button, Layout
from IPython.display import display

# Carte
from bokeh.io import curdoc
from bokeh.layouts import layout, column
from bokeh.models import ColumnDataSource, Div
from bokeh.models.tools import HoverTool
from bokeh.models.widgets import Slider, Select, TextInput
from bokeh.plotting import figure, show, output_file
import ipywidgets as widgets


In [2]:
#### Chargement des données ####

with zipfile.ZipFile('./whats-cooking-data.zip', 'r') as z:
    
    # cuisines
    file = z.open('cuisines.data', 'r')
    cuisines = pickle.load(file)
    file.close()
    
    # dfs
    file = z.open('dfs.data', 'r')
    dfs = pickle.load(file)
    file.close()
    
    # id_par_points_par_pays
    file = z.open('id_par_points_par_pays.data', 'r')
    id_par_points_par_pays = pickle.load(file)
    file.close()
    
    # id_plats_train
    file = z.open('id_plats_train.data', 'r')
    id_plats_train = pickle.load(file)
    file.close()
    
    # ingredients
    file = z.open('ingredients.data', 'r')
    ingredients = pickle.load(file)
    file.close()
    
    # ingredients_par_points_par_pays
    file = z.open('ingredients_par_points_par_pays.data', 'r')
    ingredients_par_points_par_pays = pickle.load(file)
    file.close()
    
    # model
    file = z.open('model.data', 'r')
    model = pickle.load(file)
    file.close()
    
    # neigh
    file = z.open('neigh.data', 'r')
    neigh = pickle.load(file)
    file.close()
    
    # points_par_pays
    file = z.open('points_par_pays.data', 'r')
    points_par_pays = pickle.load(file)
    file.close()
    
    # points_plats
    file = z.open('points_plats.data', 'r')
    points_plats = pickle.load(file)
    file.close()
    
z.close()


/home/julien/anaconda3/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator NMF from version 0.21.3 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/julien/anaconda3/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator NearestNeighbors from version 0.21.3 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [3]:
#### genere des couleurs aléatoires ####
def couleur_generer():
    number_of_colors = 20
    color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
             for i in range(number_of_colors)]
    return color


In [4]:
#### affichage de nos recettes sur une carte ####
#retourne la source de tous nos points afin de les afficher 
def ajouter(tout_les_points):
    color = couleur_generer()
    #on crée d'abord notre source à affichera avec un dataframe 
    table = []
    tmp = dict()
    for i in range (len(tout_les_points)):
        mes_points = np.asarray(tout_les_points[i])
        for j in range(len(mes_points)):
            tmp['color'] = color[i];
            tmp['cuisine']= cuisines[i]
            tmp = dict()
            tmp['x'] = mes_points[j][1]
            tmp['y'] = mes_points[j][0]
            tmp['id'] = id_par_points_par_pays[i][j]
            tmp['ingredient']=ingredients_par_points_par_pays[i][j].replace(", ", "\n")
            table.append(tmp)

    df_c = pd.DataFrame(table)
    return df_c.fillna(0)

In [5]:
#affiche les points de tous nos points
def afficherCarte(dataSource):
    #on prépare la carte à afficher 
    TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"
    p = figure(tools=TOOLS,  tooltips = [ ('id', '@id'),('cuisine', '@cuisine'), ('ingrédients', '<pre>@ingredient</pre>')])
    p.title.text = 'Carte des plats par couleur de pays'
    source = ColumnDataSource(dataSource) #on installe notre source pour l'afficher 
    p.circle( 'x' , 'y', source= source, fill_color= 'color' , fill_alpha= 1, line_color=None, size=10 )
    show(p)

In [6]:
#Nécessite la source de tous nos points pour mettre en valeur un de ces points 
def mettre_en_valeur_recette(dataSource, monPoint):
    df_test = dataSource.loc[dataSource['id'] == monPoint]
    return df_test
#affichage de nos points + le point qu'on veut mettre en valeur dans un graphe
def afficherCarte2(DS_tout_les_points, DS_monPoint):
    TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"
    p = figure(tools=TOOLS,  tooltips = [ ('id', '@id'),('cuisine', '@cuisine'), ('ingrédients', '<pre>@ingredient</pre>')])
    p.title.text = 'Carte des plats par couleur de pays'
    source = ColumnDataSource(DS_tout_les_points) #on installe notre source pour l'afficher 
    p.circle( 'x' , 'y', source= source, fill_color= 'color' , fill_alpha= 1, line_color=None, size=10 )
    source2 = ColumnDataSource(DS_monPoint)
    p.circle( 'x' , 'y', source= source2, fill_color= 'color' , fill_alpha= 1, line_color=None, size=30)
    show(p)

In [7]:
#affichage d'un tout nouveau point avec ses coordonnées en parametre 
def afficherNouveauPoint(DS_tout_les_points, coordonnee): #[50 ,20]
    TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"
    p = figure(tools=TOOLS,  tooltips = [ ('id', '@id'),('cuisine', '@cuisine'),('ingrédients', '<pre>@ingredient</pre>')])
    p.title.text = 'Carte des plats par couleur de pays'
    source = ColumnDataSource(DS_tout_les_points) #on installe notre source pour l'afficher 
    p.circle( 'x' , 'y', source= source, fill_color= 'color' , fill_alpha= 1, line_color=None, size=10 )
    p.circle( coordonnee[0] , coordonnee[1],  fill_color= 'red' , fill_alpha= 1, line_color=None, size=30)
    show(p)

In [8]:
def dis_moi_qui_me_ressemble_le_plus(vecteur):
    voisins = neigh.kneighbors([vecteur], return_distance=False)[0]
    cuisines_voisines = []
    test = []
    for voisin in voisins:
        cuisines_voisines.append(dfs.loc[dfs['id'] == id_plats_train[voisin]]['cuisine'].unique()[0])
        test.append(id_plats_train[voisin])
    #print(cuisines_voisines, voisins, test)
    occurence_count = Counter(cuisines_voisines)
    return occurence_count.most_common(1)[0][0]

In [9]:
def donne_moi_les_coo_de_ma_nouvelle_recette_en_6D(recette):
    recette_csv = []
    for ingr in ingredients:
        tmp = dict()
        tmp['ingredient'] = ingr
        tmp['val'] = 1 if ingr in recette.value else 0
        tmp['id'] = 'Nouvelle recette'
        recette_csv.append(tmp)

    recette_df = pd.DataFrame(recette_csv)
    recette_util_df = pd.pivot_table(data = recette_df,
                                              values = 'val', index = 'id', columns = 'ingredient')
    return model.transform(recette_util_df)[0]

def donne_moi_les_coo_de_ma_nouvelle_recette_en_6D_no_value(recette):
    recette_csv = []
    for ingr in ingredients:
        tmp = dict()
        tmp['ingredient'] = ingr
        tmp['val'] = 1 if ingr in recette else 0
        tmp['id'] = 'Nouvelle recette'
        recette_csv.append(tmp)

    recette_df = pd.DataFrame(recette_csv)
    recette_util_df = pd.pivot_table(data = recette_df,
                                              values = 'val', index = 'id', columns = 'ingredient')
    return model.transform(recette_util_df)[0]


In [10]:
def donne_moi_les_coo_de_ma_nouvelle_recette_en_2D(vecteur):
    voisins = neigh.kneighbors([vecteur], return_distance=False)[0]
    cuisines_voisines = []
    coo = []
    for voisin in voisins:
        cuisines_voisines.append(dfs.loc[dfs['id'] == id_plats_train[voisin]]['cuisine'].unique()[0])
        coo.append(points_plats[voisin])
    return np.mean(coo, axis=0)


In [17]:
#### Construction de la carte des recettes ####

df_c = ajouter(points_par_pays)


In [18]:
#### Affichage de la carte des recettes ####

afficherCarte(df_c)


In [19]:
nouvelle_recette = widgets.SelectMultiple(
    options=ingredients,
    #value=['milk', 'honey', 'sweet potatoes'],
    value=['mozzarella cheese', 'mutton', 'olives'],
    description='Ingrédients :',
    disabled=False,
    layout = Layout(width='650px', height='200px'),
)
display(nouvelle_recette)


SelectMultiple(description='Ingrédients :', index=(4194, 4230, 4349), layout=Layout(height='200px', width='650…

In [20]:
#### Récupération des vecteurs 6 et 2 dimentions

nouvelle_recette_W = donne_moi_les_coo_de_ma_nouvelle_recette_en_6D(nouvelle_recette)
coordonnee = donne_moi_les_coo_de_ma_nouvelle_recette_en_2D(nouvelle_recette_W)

# Récapitulatif du nouveau plat

print(nouvelle_recette.value)


('mozzarella cheese', 'mutton', 'olives')


In [21]:
#print(nouvelle_recette_W)
print(dis_moi_qui_me_ressemble_le_plus(nouvelle_recette_W))


italian


In [22]:
afficherNouveauPoint(df_c, coordonnee)


In [37]:
sortie = open("results.csv", "w+")

with open('test.json', 'r') as f:
    datas = json.load(f)
total = len(datas)
fait = 0
for data in datas:
    data6D = donne_moi_les_coo_de_ma_nouvelle_recette_en_6D_no_value(data['ingredients'])
    id = str(data['id'])
    cuisine = dis_moi_qui_me_ressemble_le_plus(data6D)
    line = id + "," + cuisine + "\n"
    sortie.write(line)
    fait += 1
    print((fait/total)*100, "%")
sortie.close()

0.01005631536604988 %
0.02011263073209976 %
0.030168946098149636 %
0.04022526146419952 %
0.0502815768302494 %
0.06033789219629927 %
0.07039420756234915 %
0.08045052292839903 %
0.09050683829444892 %
0.1005631536604988 %
0.11061946902654868 %
0.12067578439259855 %
0.13073209975864844 %
0.1407884151246983 %
0.15084473049074817 %
0.16090104585679807 %
0.17095736122284796 %
0.18101367658889783 %
0.1910699919549477 %
0.2011263073209976 %
0.21118262268704746 %
0.22123893805309736 %
0.23129525341914725 %
0.2413515687851971 %
0.251407884151247 %
0.2614641995172969 %
0.2715205148833467 %
0.2815768302493966 %
0.2916331456154465 %
0.30168946098149635 %
0.31174577634754624 %
0.32180209171359614 %
0.33185840707964603 %
0.34191472244569593 %
0.35197103781174577 %
0.36202735317779566 %
0.37208366854384556 %
0.3821399839098954 %
0.3921962992759453 %
0.4022526146419952 %
0.4123089300080451 %
0.4223652453740949 %
0.4324215607401448 %
0.4424778761061947 %
0.4525341914722446 %
0.4625905068382945 %
0.472646

3.982300884955752 %
3.992357200321802 %
4.002413515687852 %
4.0124698310539015 %
4.022526146419952 %
4.032582461786002 %
4.042638777152051 %
4.052695092518102 %
4.062751407884151 %
4.072807723250201 %
4.082864038616251 %
4.092920353982301 %
4.102976669348351 %
4.113032984714401 %
4.1230893000804505 %
4.133145615446501 %
4.14320193081255 %
4.1532582461786 %
4.1633145615446505 %
4.1733708769107 %
4.18342719227675 %
4.1934835076428 %
4.20353982300885 %
4.2135961383749 %
4.223652453740949 %
4.2337087691069994 %
4.243765084473049 %
4.253821399839099 %
4.263877715205148 %
4.273934030571199 %
4.283990345937248 %
4.294046661303298 %
4.304102976669348 %
4.314159292035398 %
4.324215607401448 %
4.334271922767498 %
4.3443282381335475 %
4.354384553499598 %
4.364440868865647 %
4.374497184231697 %
4.3845534995977475 %
4.394609814963797 %
4.404666130329847 %
4.414722445695897 %
4.424778761061947 %
4.434835076427997 %
4.444891391794046 %
4.4549477071600965 %
4.465004022526147 %
4.475060337892196 %
4.48

8.105390185036203 %
8.115446500402253 %
8.125502815768302 %
8.135559131134352 %
8.145615446500402 %
8.155671761866452 %
8.165728077232503 %
8.175784392598553 %
8.185840707964601 %
8.195897023330652 %
8.205953338696702 %
8.216009654062752 %
8.226065969428802 %
8.23612228479485 %
8.246178600160901 %
8.256234915526951 %
8.266291230893001 %
8.276347546259052 %
8.2864038616251 %
8.29646017699115 %
8.3065164923572 %
8.31657280772325 %
8.326629123089301 %
8.336685438455351 %
8.3467417538214 %
8.35679806918745 %
8.3668543845535 %
8.37691069991955 %
8.3869670152856 %
8.397023330651649 %
8.4070796460177 %
8.41713596138375 %
8.4271922767498 %
8.43724859211585 %
8.447304907481898 %
8.457361222847949 %
8.467417538213999 %
8.47747385358005 %
8.487530168946098 %
8.497586484312148 %
8.507642799678198 %
8.517699115044246 %
8.527755430410297 %
8.537811745776347 %
8.547868061142397 %
8.557924376508447 %
8.567980691874496 %
8.578037007240546 %
8.588093322606596 %
8.598149637972647 %
8.608205953338697 %
8.

12.168141592920353 %
12.178197908286403 %
12.188254223652454 %
12.198310539018504 %
12.208366854384552 %
12.218423169750602 %
12.228479485116653 %
12.238535800482703 %
12.248592115848753 %
12.258648431214802 %
12.268704746580852 %
12.278761061946902 %
12.288817377312952 %
12.298873692679003 %
12.308930008045053 %
12.318986323411101 %
12.329042638777151 %
12.339098954143202 %
12.349155269509252 %
12.359211584875302 %
12.36926790024135 %
12.3793242156074 %
12.389380530973451 %
12.399436846339501 %
12.409493161705552 %
12.419549477071602 %
12.42960579243765 %
12.4396621078037 %
12.44971842316975 %
12.4597747385358 %
12.469831053901851 %
12.4798873692679 %
12.48994368463395 %
12.5 %
12.51005631536605 %
12.5201126307321 %
12.530168946098149 %
12.5402252614642 %
12.55028157683025 %
12.5603378921963 %
12.57039420756235 %
12.580450522928398 %
12.590506838294448 %
12.600563153660499 %
12.610619469026549 %
12.6206757843926 %
12.63073209975865 %
12.640788415124698 %
12.650844730490748 %
12.660901

16.140386162510055 %
16.150442477876105 %
16.160498793242155 %
16.170555108608205 %
16.180611423974256 %
16.190667739340306 %
16.200724054706356 %
16.210780370072406 %
16.220836685438456 %
16.230893000804507 %
16.240949316170557 %
16.251005631536604 %
16.261061946902654 %
16.271118262268704 %
16.281174577634754 %
16.291230893000805 %
16.301287208366855 %
16.311343523732905 %
16.321399839098955 %
16.331456154465005 %
16.341512469831056 %
16.351568785197106 %
16.361625100563153 %
16.371681415929203 %
16.381737731295253 %
16.391794046661303 %
16.401850362027353 %
16.411906677393404 %
16.421962992759454 %
16.432019308125504 %
16.442075623491554 %
16.452131938857605 %
16.462188254223655 %
16.4722445695897 %
16.48230088495575 %
16.492357200321802 %
16.502413515687852 %
16.512469831053902 %
16.522526146419953 %
16.532582461786003 %
16.542638777152053 %
16.552695092518103 %
16.562751407884154 %
16.5728077232502 %
16.58286403861625 %
16.5929203539823 %
16.60297666934835 %
16.6130329847144 %
16.

...

81.52654867256636 %
81.53660498793242 %
81.54666130329848 %
81.55671761866452 %
81.56677393403056 %
81.57683024939662 %
81.58688656476268 %
81.59694288012872 %
81.60699919549477 %
81.61705551086082 %
81.62711182622687 %
81.63716814159292 %
81.64722445695897 %
81.65728077232502 %
81.66733708769107 %
81.67739340305712 %
81.68744971842317 %
81.69750603378922 %
81.70756234915527 %
81.71761866452132 %
81.72767497988737 %
81.73773129525343 %
81.74778761061947 %
81.75784392598551 %
81.76790024135157 %
81.77795655671763 %
81.78801287208367 %
81.79806918744971 %
81.80812550281577 %
81.81818181818183 %
81.82823813354787 %
81.83829444891391 %
81.84835076427997 %
81.85840707964603 %
81.86846339501207 %
81.87851971037811 %
81.88857602574416 %
81.89863234111023 %
81.90868865647627 %
81.91874497184232 %
81.92880128720836 %
81.93885760257442 %
81.94891391794047 %
81.95897023330652 %
81.96902654867256 %
81.97908286403862 %
81.98913917940467 %
81.99919549477072 %
82.00925181013676 %
82.01930812550282 %


85.68986323411102 %
85.69991954947707 %
85.70997586484313 %
85.72003218020917 %
85.73008849557522 %
85.74014481094126 %
85.75020112630732 %
85.76025744167337 %
85.77031375703942 %
85.78037007240546 %
85.79042638777152 %
85.80048270313758 %
85.81053901850362 %
85.82059533386966 %
85.83065164923572 %
85.84070796460178 %
85.85076427996782 %
85.86082059533386 %
85.87087691069992 %
85.88093322606596 %
85.89098954143202 %
85.90104585679806 %
85.91110217216412 %
85.92115848753016 %
85.93121480289622 %
85.94127111826226 %
85.95132743362832 %
85.96138374899436 %
85.97144006436042 %
85.98149637972647 %
85.99155269509252 %
86.00160901045857 %
86.01166532582461 %
86.02172164119067 %
86.03177795655672 %
86.04183427192277 %
86.05189058728881 %
86.06194690265487 %
86.07200321802092 %
86.08205953338697 %
86.09211584875301 %
86.10217216411907 %
86.11222847948513 %
86.12228479485117 %
86.13234111021721 %
86.14239742558325 %
86.15245374094933 %
86.16251005631537 %
86.17256637168141 %
86.18262268704746 %


89.84312148028962 %
89.85317779565567 %
89.86323411102171 %
89.87329042638777 %
89.88334674175383 %
89.89340305711987 %
89.90345937248591 %
89.91351568785197 %
89.92357200321803 %
89.93362831858407 %
89.94368463395011 %
89.95374094931617 %
89.96379726468223 %
89.97385358004827 %
89.98390989541431 %
89.99396621078037 %
90.00402252614641 %
90.01407884151247 %
90.02413515687851 %
90.03419147224457 %
90.04424778761062 %
90.05430410297667 %
90.06436041834272 %
90.07441673370877 %
90.08447304907482 %
90.09452936444087 %
90.10458567980692 %
90.11464199517297 %
90.12469831053902 %
90.13475462590506 %
90.14481094127112 %
90.15486725663717 %
90.16492357200322 %
90.17497988736926 %
90.18503620273532 %
90.19509251810138 %
90.20514883346742 %
90.21520514883346 %
90.22526146419952 %
90.23531777956558 %
90.24537409493162 %
90.25543041029766 %
90.2654867256637 %
90.27554304102978 %
90.28559935639582 %
90.29565567176186 %
90.3057119871279 %
90.31576830249396 %
90.32582461786002 %
90.33588093322606 %
90

94.00643604183428 %
94.01649235720032 %
94.02654867256636 %
94.03660498793242 %
94.04666130329848 %
94.05671761866452 %
94.06677393403056 %
94.07683024939662 %
94.08688656476268 %
94.09694288012872 %
94.10699919549477 %
94.11705551086082 %
94.12711182622687 %
94.13716814159292 %
94.14722445695897 %
94.15728077232502 %
94.16733708769107 %
94.17739340305712 %
94.18744971842317 %
94.19750603378922 %
94.20756234915527 %
94.21761866452132 %
94.22767497988737 %
94.23773129525343 %
94.24778761061947 %
94.25784392598551 %
94.26790024135157 %
94.27795655671763 %
94.28801287208367 %
94.29806918744971 %
94.30812550281577 %
94.31818181818183 %
94.32823813354787 %
94.33829444891391 %
94.34835076427997 %
94.35840707964603 %
94.36846339501207 %
94.37851971037811 %
94.38857602574416 %
94.39863234111023 %
94.40868865647627 %
94.41874497184232 %
94.42880128720836 %
94.43885760257442 %
94.44891391794047 %
94.45897023330652 %
94.46902654867256 %
94.47908286403862 %
94.48913917940467 %
94.49919549477072 %


98.16975060337893 %
98.17980691874497 %
98.18986323411102 %
98.19991954947707 %
98.20997586484313 %
98.22003218020917 %
98.23008849557522 %
98.24014481094126 %
98.25020112630732 %
98.26025744167337 %
98.27031375703942 %
98.28037007240546 %
98.29042638777152 %
98.30048270313758 %
98.31053901850362 %
98.32059533386966 %
98.33065164923572 %
98.34070796460178 %
98.35076427996782 %
98.36082059533386 %
98.37087691069992 %
98.38093322606596 %
98.39098954143202 %
98.40104585679806 %
98.41110217216412 %
98.42115848753016 %
98.43121480289622 %
98.44127111826226 %
98.45132743362832 %
98.46138374899436 %
98.47144006436042 %
98.48149637972647 %
98.49155269509252 %
98.50160901045857 %
98.51166532582461 %
98.52172164119067 %
98.53177795655672 %
98.54183427192277 %
98.55189058728881 %
98.56194690265487 %
98.57200321802092 %
98.58205953338697 %
98.59211584875301 %
98.60217216411907 %
98.61222847948513 %
98.62228479485117 %
98.63234111021721 %
98.64239742558325 %
98.65245374094933 %
98.66251005631537 %
